In [1]:
from pyspark.sql import SparkSession

# Kubernetes dashboard > Config And Storage > Secrets > s3a-env-configuration
#http://192.168.29.16:9090/browser/test login() :- E2XLJVLJHRZVUVLR & ZQ5BDKSIGDTIUI0CSRZYW22DRWKCTZLE

#Kubernetes dashboard > Service > Services > 
#Name (s3a-hl) 
#Labels(v1.min.io/tenant: s3a) Type(ClusterIP) Cluster IP(none) and Internal Endpoints(s3a-hl.data-platform:9000 TCP & s3a-hl.data-platform:0 TCP)
#http://s3a-hl.data-platform:9000

#http://192.168.29.16:9090 go to Minio console then create Object Browser and upload csv file and then create Access Keys
# "accessKey":"qUqE7TK7Xhkw9hS8DS4b","secretKey":"MeJYIY6s4X8P0SjYF7AsFkgf9AePLu837gndpAKS"

minio_endpoint = "http://s3a-hl.data-platform:9000"  # Ensure this matches your setup
minio_access_key = "qUqE7TK7Xhkw9hS8DS4b"
minio_secret_key = "MeJYIY6s4X8P0SjYF7AsFkgf9AePLu837gndpAKS"
minio_bucket = "test"

spark = SparkSession.builder \
    .appName("MinIOs") \
    .config('spark.master','local')\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()


In [2]:
dataframe = spark.read.json('s3a://test/orders.json')

average = dataframe.agg({'amount':'avg'})

#average.write.format("csv").option("header", "true").save("s3a://test/json/")

average.show()

+-----------+
|avg(amount)|
+-----------+
|     2500.0|
+-----------+



In [3]:
df = spark.read.csv("s3a://test/taxi.csv")
df.show()
spark.stop()

+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+
|     _c0|                 _c1|                 _c2|            _c3|          _c4|       _c5|               _c6|         _c7|         _c8|         _c9|       _c10| _c11|   _c12|      _c13|        _c14|                _c15|        _c16|
+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_date...|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surch...|total_amount|
|       2|02/08/2018 05:07:...|02/08/2018 05:16:...|    